## TÍNH TOÁN VÀ VẼ BIỂU ĐỒ TRÒN CỦA TRANG BỊ TỘC VÀ HỆ

In [ ]:
import pandas as pd
import numpy as np
import ast
from itertools import chain

### ĐỌC DỮ LIỆU ĐỂ LẤY TẤT CẢ CÁC MÓN ĐỒ

In [ ]:
item_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols = ['List Name Item']).drop_duplicates().reset_index(drop=True)

### LẤY RA SỐ LƯỢNG ĐỒ CÓ TRONG TRẬN ĐẤU

ast.literak_eval: dùng để biến đổi string thành list

In [ ]:
list_items = []
for i in range(len(item_df['List Name Item'])):
    list_items.append(ast.literal_eval(item_df['List Name Item'][i])) 

### TÌM RA MÓN ĐỒ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

hàm zip dùng để biến 2 list được cho ra thành key và values

lisst(chain) dùng để ép mảng xuống còn 1D và sau đó sort lại

In [ ]:
def get_best_worst_items(list_items):
    flatten_list = list(chain.from_iterable(list_items))
    flatten_list = list(chain.from_iterable(flatten_list))
    flatten_list = np.array(flatten_list, dtype = object)
    unique, counts = np.unique(flatten_list, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_item = list(b.items())
    worst_item = list(b.items())[0:5]
    return best_item,worst_item

In [ ]:
get_best_worst_items(list_items)

### TÌM VỊ TRÍ TRUNG BÌNH CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU


khi người chơi sử dụng món đồ đó thì sẽ được lọt vào top mấy của game

In [ ]:
average_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols = ['Puuid', 'Placement', 'List Name Item']).drop_duplicates().reset_index(drop = True)

In [ ]:
def convert_string_to_list(ok, placement, column):
    temp = ok[ok.eq(placement).any(1)].reset_index(drop=True)
    top = []
    for i in range(len(temp)):
        top.append(ast.literal_eval(temp[column][i]))
    return top

lọc ra trong những top từ 1 đến 8 số lần item đó được sử dụng

In [ ]:
item_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(average_df, i, 'List Name Item')
    temp1 = 'top_' + str(i)
    item_on_top[temp1] = get_best_worst_items(temp)

In [ ]:
item_on_top

### TỈ LỆ CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA MÓN ĐỒ TRONG TỪNG TOP CỦA TRẬN ĐẤU

In [ ]:
all_item_df = pd.read_csv("item_tft_csv.csv", usecols=["Item"])

In [ ]:
all_items = all_item_df['Item'].to_list()

temp: vị trí trung bình của món đồ khi ở trong top từ 1 đến 8

temp1: tỉ lệ số trận đấu từ top 1 đến 8 mà món đồ đó được sử dụng

In [ ]:
sum = 0
temp = 0
item_avg_top = []
item_avg_on_match = []
match_each_top = []
for item in all_items:
    temp1 = []
    temp2 = []
    for i in range(1,9,1):
        cc = dict(item_on_top['top_' + str(i)][0])
        if(item in cc):
            temp = temp+dict(item_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(item_on_top['top_' + str(i)][0])[item])
            temp1.append(temp/sum)
            temp2.append(sum/32000)
            match_each_top.append(sum)
    item_avg_top.append(temp1)
    item_avg_on_match.append(temp2)
    temp = 0
    sum = 0

In [ ]:
item_avg_top

avg_top: phần trăm của món đồ

avg_on_match: phần trăm món đồ đó được sử dụng trong mỗi hạng(1->8)

## TƯƠNG TỰ VỚI TỘC VÀ HỆ

#### đọc dữ liệu để lấy hệ và tộc

In [ ]:
traits_df = pd.read_csv('process_data_tft_general_csv.csv', usecols = ['List Traits']).drop_duplicates().reset_index(drop=True)

In [ ]:
list_traits = []

for i in range(len(traits_df['List Traits'])):
    list_traits.append(ast.literal_eval(traits_df['List Traits'][i])) 

### TÌM RA TỘC HỆ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

In [ ]:
def get_best_worst_traits(list_traits):
    flatten_list = list(chain.from_iterable(list_traits))
    flatten_list = np.array(flatten_list, dtype = object)
    cc = flatten_list[0::3]
    unique, counts= np.unique(cc, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_traits = list(b.items())
    worst_traits = list(b.items())[:5]
    return best_traits, worst_traits

In [ ]:
a, b = get_best_worst_traits(list_traits)
print(a, b)

### VỊ TRÍ TRUNG BÌNH CỦA TỘC VÀ HỆ

In [ ]:
list_traits_df = pd.read_csv('process_data_tft_general_csv.csv', usecols= ['Placement', 'List Traits'])

In [ ]:
traits_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(list_traits_df, i, 'List Traits')
    temp1 = 'top_' + str(i)
    traits_on_top[temp1] = get_best_worst_traits(temp)

### TỈ LỆ CỦA TỘC HỆ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA TỘC HỆ TRONG TỪNG TOP CỦA TRẬN ĐẤU

traits_avg_top: tộc hệ trung bình trong các top trận đấu

traits_avg_on_match: tần suất xuất hiện của tộc hệ trong trận đấu

In [ ]:
all_traits_df = pd.read_csv('TocHe.csv', usecols=['Trait'])

In [ ]:
all_traits = all_traits_df['Trait'].to_list()

In [ ]:
sum = 0
temp = 0
traits_avg_top = []
traits_avg_on_match = []
traits_match_each_top = []
for item in all_traits:
    temp1 = []
    temp2 = []
    for i in range(1,9,1):
        cc = dict(traits_on_top['top_' + str(i)][0])
        if(item in cc):
            temp = temp+dict(traits_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(traits_on_top['top_' + str(i)][0])[item])
            temp1.append(temp/sum)
            temp2.append(sum/32000)
            traits_match_each_top.append(sum)
    traits_avg_top.append(temp1)
    traits_avg_on_match.append(temp2)
    temp = 0
    sum = 0

In [ ]:
print(traits_avg_top,all_traits, traits_avg_on_match)